In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,GlobalAveragePooling1D,Embedding
from tensorflow.keras.preprocessing.text import Tokenizer


In [2]:
corpus=[]
with open("LP-IV-datasets/CBOW(Ass5)/CBOW.txt",'r') as f:
    for line in f:
        corpus.append(line.strip())

In [5]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(corpus)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(corpus)

X,y=[],[]
for seq in sequences:
    for i,target_word in enumerate(seq):
        for j in range(max(0,i-2),min(len(seq),i+3)):
            if i!=j:
                X.append([target_word,seq[j]])
                y.append(1)
X=np.array(X)
y=np.array(y)

In [7]:
model=Sequential([
    Embedding(input_dim=len(word_index)+1,output_dim=1,input_length=2),
    GlobalAveragePooling1D(),
    Dense(1,activation="sigmoid")
])
model.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=0)

In [8]:
target_word="interval"

if target_word not in word_index:
    print("Bro")
else:
    target_embeddings=model.layers[0].get_weights()[0][word_index[target_word]]
    
    def cosine_sim(a,b):
        return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    similarities=[(word,cosine_sim(target_embeddings,model.layers[0].get_weights()[0][index]))
                 for word,index in word_index.items() if word != target_word]
    related_words=sorted(similarities,key=lambda x:x[1],reverse=True)[:5]
    print(f"Word related to {target_word} are ,{[word for word,_ in related_words]}")

Word related to interval are ,['the', 'of', 'influenza', 'covid', '19']
